# Scraping mine data with `.apply`

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Doing this programmatically

## First, scraping a single page

### Import your imports

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
import time

### Searching for a mine

Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm) and use Selenium to search for `3503598`

- *TIP: You might need to use the Selenium code to scroll down to the right spot on the page. Or not!*
- *TIP: Use `.send_keys` to type into the box*
- *TIP: On pages that never change, you can usually just use XPath if you're feeling lazy*

In [15]:
driver = webdriver.Chrome()
driver.get('https://arlweb.msha.gov/drs/drshome.htm')

In [23]:
# to scroll down
text_input = driver.find_elements_by_id('inputdrs')[1]
driver.execute_script("arguments[0].scrollIntoView(true)", text_input)
text_input.send_keys('3503598')
# click search
driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input').click()


### Finding reports

On the "Report Selection Page" (where you should be after you search), use Selenium to...

- Change the **Beginning Date** to `1/1/1995`
- Select the report type of `Violations`
- Click **Get Report**

.

- *TIP: Remember, if someone isn't on the page Selenium can't click it!*

In [24]:
driver.find_element_by_name('BDate').send_keys('1/1/1995')

In [25]:
driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input').click()


In [26]:
driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input').click()

### Saving reports

Save all of the rows of data on that page into a new dataframe. Each column is its own column, **and you also need to save the URL under the 'Standard' column.** Here, I even made you a blank dictionary:

```python
data = {}
data['violator'] = ''
data['contract_id'] = ''
data['citation_no'] = ''
data['case_no'] = ''
data['date_issues'] = ''
data['final_order_date'] = ''
data['section_of_act'] = ''
data['date_terminated'] = ''
data['citation'] = ''
data['s_and_s'] = ''
data['standard'] = ''
data['standard_url'] = ''
data['proposed_penalty'] = ''
data['citation_status'] = ''
data['current_penalty'] = ''
data['amount_paid'] = ''
```

- *TIP: Some of those table rows aren't what you want. How can you tell them apart from the good ones? (the previous mine owner ones are okay, I just mean the weird headers)*
- *TIP: I sense `.find_elements` + a lot of square brackets*
- *TIP: This is just like scraping a search results page!*
- *TIP: For the URL, you'll need to find the `a` inside of the cell*
- *TIP: class name is sadly not going to save your life here, because some of the `tr`s and `td`s have the same class! It's stupid. But there's a trick: CSS selectors! Something like `div#container` finds a `div` with the id of `container`, while `span.important` finds a `span` with the class of `important`. It should be helpful! And use `.find_elements_by_` + tab to see what the command is*

In [66]:
# try to get the first row's info
first_row = driver.find_elements_by_css_selector('tr.drsviols')[0]
violator = first_row.find_elements_by_tag_name('td')[0]
print(violator.text)
contract_id = first_row.find_elements_by_tag_name('td')[1]
print(contract_id.text)
# So, they are all in order

Newberg Rock & Dirt
  


In [67]:
data_list = []

rows = driver.find_elements_by_css_selector('tr.drsviols') 

for row in rows:
    data = {}
    data['violator'] = row.find_elements_by_tag_name('td')[0].text
    data['contract_id'] = row.find_elements_by_tag_name('td')[1].text
    data['citation_no'] = row.find_elements_by_tag_name('td')[2].text
    data['case_no'] = row.find_elements_by_tag_name('td')[3].text
    data['date_issues'] = row.find_elements_by_tag_name('td')[4].text
    data['final_order_date'] = row.find_elements_by_tag_name('td')[5].text
    data['section_of_act'] = row.find_elements_by_tag_name('td')[6].text
    data['date_terminated'] = row.find_elements_by_tag_name('td')[7].text
    data['citation'] = row.find_elements_by_tag_name('td')[8].text
    data['s_and_s'] = row.find_elements_by_tag_name('td')[9].text
    
    standard = row.find_elements_by_tag_name('td')[10]
    data['standard'] = standard.text
    data['standard_url'] = standard.find_element_by_tag_name('a').get_attribute('href')
    
    data['proposed_penalty'] = row.find_elements_by_tag_name('td')[11].text
    data['citation_status'] = row.find_elements_by_tag_name('td')[12].text
    data['current_penalty'] = row.find_elements_by_tag_name('td')[13].text
    data['amount_paid'] = row.find_elements_by_tag_name('td')[14].text
    
    data_list.append(data)
    
data_list


[{'violator': 'Newberg Rock & Dirt',
  'contract_id': '  ',
  'citation_no': '8992368  ',
  'case_no': '000440809',
  'date_issues': '5/2/2017',
  'final_order_date': '7/23/2017 ',
  'section_of_act': '104(a)',
  'date_terminated': '5/2/2017',
  'citation': 'C',
  's_and_s': 'N',
  'standard': '56.14107(a)',
  'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf',
  'proposed_penalty': '116.00',
  'citation_status': 'Closed',
  'current_penalty': '116.00 ',
  'amount_paid': '116.00 '},
 {'violator': 'Newberg Rock & Dirt',
  'contract_id': '  ',
  'citation_no': '8992214  ',
  'case_no': '000435717',
  'date_issues': '2/7/2017',
  'final_order_date': '5/21/2017 ',
  'section_of_act': '104(a)',
  'date_terminated': '2/8/2017',
  'citation': 'C',
  's_and_s': 'N',
  'standard': '56.14107(a)',
  'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf',
  'proposed_penalty': '116.0

### Saving that data

Save the dataframe to a CSV file called `3503598-violations.csv` (that's the TDLR code)

In [68]:
df = pd.DataFrame(data_list)
df.head()

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,116.00,000440809,C,8992368,Closed,,116.00,5/2/2017,5/2/2017,7/23/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
1,116.00,000435717,C,8992214,Closed,,116.00,2/7/2017,2/8/2017,5/21/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
2,100.00,000398910,C,8872462,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
3,100.00,000398910,C,8872461,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
4,100.00,000383840,C,8790965,Closed,,100.00,4/15/2015,4/15/2015,7/17/2015,100.00,N,104(a),56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt


In [69]:
df.to_csv('3503598-violations.csv',index=False)


## Put that all in ONE cell that runs correctly

The **entire process**, from searching to saving as a CSV

In [71]:
driver = webdriver.Chrome()
driver.get('https://arlweb.msha.gov/drs/drshome.htm')
# to scroll down
text_input = driver.find_elements_by_id('inputdrs')[1]
driver.execute_script("arguments[0].scrollIntoView(true)", text_input)
text_input.send_keys('3503598')
# click search
driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input').click()

# get to the search page
time.sleep(1)
# enter beginning date
driver.find_element_by_name('BDate').send_keys('1/1/1995')
# click "violation"
driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input').click()
# get report
driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input').click()

# scraping!
data_list = []

rows = driver.find_elements_by_css_selector('tr.drsviols') 

for row in rows:
    data = {}
    data['violator'] = row.find_elements_by_tag_name('td')[0].text
    data['contract_id'] = row.find_elements_by_tag_name('td')[1].text
    data['citation_no'] = row.find_elements_by_tag_name('td')[2].text
    data['case_no'] = row.find_elements_by_tag_name('td')[3].text
    data['date_issues'] = row.find_elements_by_tag_name('td')[4].text
    data['final_order_date'] = row.find_elements_by_tag_name('td')[5].text
    data['section_of_act'] = row.find_elements_by_tag_name('td')[6].text
    data['date_terminated'] = row.find_elements_by_tag_name('td')[7].text
    data['citation'] = row.find_elements_by_tag_name('td')[8].text
    data['s_and_s'] = row.find_elements_by_tag_name('td')[9].text
    
    standard = row.find_elements_by_tag_name('td')[10]
    data['standard'] = standard.text
    data['standard_url'] = standard.find_element_by_tag_name('a').get_attribute('href')
    
    data['proposed_penalty'] = row.find_elements_by_tag_name('td')[11].text
    data['citation_status'] = row.find_elements_by_tag_name('td')[12].text
    data['current_penalty'] = row.find_elements_by_tag_name('td')[13].text
    data['amount_paid'] = row.find_elements_by_tag_name('td')[14].text
    
    data_list.append(data)
    
print(data_list)

# trun into dataframe
df = pd.DataFrame(data_list)

# save it to csv file
df.to_csv('3503598-violations.csv',index=False)


[{'violator': 'Newberg Rock & Dirt', 'contract_id': '  ', 'citation_no': '8992368  ', 'case_no': '000440809', 'date_issues': '5/2/2017', 'final_order_date': '7/23/2017 ', 'section_of_act': '104(a)', 'date_terminated': '5/2/2017', 'citation': 'C', 's_and_s': 'N', 'standard': '56.14107(a)', 'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf', 'proposed_penalty': '116.00', 'citation_status': 'Closed', 'current_penalty': '116.00 ', 'amount_paid': '116.00 '}, {'violator': 'Newberg Rock & Dirt', 'contract_id': '  ', 'citation_no': '8992214  ', 'case_no': '000435717', 'date_issues': '2/7/2017', 'final_order_date': '5/21/2017 ', 'section_of_act': '104(a)', 'date_terminated': '2/8/2017', 'citation': 'C', 's_and_s': 'N', 'standard': '56.14107(a)', 'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf', 'proposed_penalty': '116.00', 'citation_status': 'Closed', 'current_penalty': '11

# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [73]:
new_df = pd.read_csv('mines-subset.csv')
new_df

,id
0,4104757
1,801306
2,3609931


### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them? If so, make them match.

- *TIP: You can zero fill if you want, but another option is that when reading in a CSV, `dtype='str'` will force everything to be a string*

In [77]:
# it should be 0801306
new_df = pd.read_csv('mines-subset.csv',dtype ='str')
new_df

,id
0,4104757
1,0801306
2,3609931


### Convert your one-cell scraper into a function, and use it on each row of our dataset

- *TIP: You'll be using `.apply`*
- *TIP: You won't be joining this back into your dataframe, so you don't need to `return` anything or `join` or any of that.*
- *TIP: Be careful of your **other variable names** - if you're calling the thing you're sending your function `row`, you can't use it anywhere else (like in your loop)*
- *TIP: **BE CAREFUL WHAT YOU NAME YOUR DATAFRAMES.** If you name the citations dataframe `df` it can overwrite your mine ID `df`*
- *TIP: You'll be saving a dataframe each time*
- *TIP: Be sure you change everything that refers to the mine ID to refer to the current row's ID instead of `3503598`*
- *TIP: BE SURE TO CHANGE EVERYTHING THAT REFERS TO THE MINE ID*
- *TIP: EVERYTHING, EVERYTHING, EVERYTHING! Look at the end of your function! Maybe I'm overreacting, I don't know.*
- *TIP: If you hit an error about list index out of range, see what line it's happening on and go look at the page. What's different about this page than the previous ones? (answer: the last three columns!) If you assign those columns later using `try`/`except` you should be able to get some data from those rows without throwing it all out. If you can't figure it out, just wrap it all in try/except and give up on those rows*
- *TIP: Some of the standards might not have links, either, so you might want to wrap that in a `try`/`except`, too!*

In [87]:
data_list =[]

def get_violation_info(result):
    driver = webdriver.Chrome()
    driver.get('https://arlweb.msha.gov/drs/drshome.htm')
    # to scroll down
    text_input = driver.find_elements_by_id('inputdrs')[1]
    driver.execute_script("arguments[0].scrollIntoView(true)", text_input)
    text_input.send_keys(result['id'])
    # click search
    driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input').click()

    # get to the search page
    time.sleep(1)
    # enter beginning date
    driver.find_element_by_name('BDate').send_keys('1/1/1995')
    # click "violation"
    driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input').click()
    # get report
    driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input').click()

    # scraping!
    

    rows = driver.find_elements_by_css_selector('tr.drsviols') 

    for row in rows:
        data = {}
        data['violator'] = row.find_elements_by_tag_name('td')[0].text
        data['contract_id'] = row.find_elements_by_tag_name('td')[1].text
        data['citation_no'] = row.find_elements_by_tag_name('td')[2].text
        data['case_no'] = row.find_elements_by_tag_name('td')[3].text
        data['date_issues'] = row.find_elements_by_tag_name('td')[4].text
        data['final_order_date'] = row.find_elements_by_tag_name('td')[5].text
        data['section_of_act'] = row.find_elements_by_tag_name('td')[6].text
        data['date_terminated'] = row.find_elements_by_tag_name('td')[7].text
        data['citation'] = row.find_elements_by_tag_name('td')[8].text
        data['s_and_s'] = row.find_elements_by_tag_name('td')[9].text

        standard = row.find_elements_by_tag_name('td')[10]
        data['standard'] = standard.text
        try:
            data['standard_url'] = standard.find_element_by_tag_name('a').get_attribute('href')
        except:
            data['standard_url'] = 'no data'
        try:
            data['proposed_penalty'] = row.find_elements_by_tag_name('td')[11].text
        except:
            data['proposed_penalty'] ='no data'
        try:
            data['citation_status'] = row.find_elements_by_tag_name('td')[12].text
        except:
            data['citation_status']= 'no data'
        try:
            data['current_penalty'] = row.find_elements_by_tag_name('td')[13].text
        except:
            data['current_penalty'] ='no data'
        try:
            data['amount_paid'] = row.find_elements_by_tag_name('td')[14].text
        except:

            data['amount_paid']='no data'

#         except:
#             print("no data")

        data_list.append(data)
        
    return data_list
    
    #print(data_list)

# # trun into dataframe
# df = pd.DataFrame(data_list)

# # save it to csv file
# df.to_csv('3503598-violations.csv',index=False)


In [88]:
# apply new_df(only three ids) 
subset_df = new_df.apply(get_violation_info, axis=1)  #.join(new_df)
#pd.DataFrame(subset_df)

In [92]:
pd.DataFrame(subset_df)

,0
0,"[{'violator': 'Dirt Works', 'contract_id': ' ..."
1,"[{'violator': 'Dirt Works', 'contract_id': ' ..."
2,"[{'violator': 'Dirt Works', 'contract_id': ' ..."


In [93]:
pd.DataFrame(subset_df[0])

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,100.00,000374480,C,8778046,Closed,,100.00,10/14/2014,10/14/2014,3/27/2015,100.00,N,104(a),56.14132(a),http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
1,162.00,000374480,C,8778047,Closed,,162.00,10/14/2014,11/4/2014,3/27/2015,162.00,N,104(a),56.18010,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
2,100.00,000348280,C,8771781,Closed,,100.00,1/22/2014,1/22/2014,5/21/2014,100.00,N,104(a),56.14100(b),http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
3,243.00,000345454,C,8771782,Closed,,243.00,1/22/2014,1/22/2014,4/20/2014,243.00,Y,104(a),56.9300(a),http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
4,243.00,000345454,C,8771783,Closed,,243.00,1/22/2014,1/22/2014,4/20/2014,243.00,Y,104(a),56.9300(a),http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
5,100.00,000345454,C,8771784,Closed,,100.00,1/22/2014,1/22/2014,4/20/2014,100.00,N,104(a),50.30(a),http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Dirt Works
6,100.00,000316329,C,8688404,Closed,,100.00,1/29/2013,1/29/2013,4/19/2013,100.00,N,104(a),50.30(a),http://www.gpo.gov/fdsys/pkg/CFR-2013-title30-...,Dirt Works
7,100.00,000316329,C,8681997,Closed,,100.00,1/28/2013,2/1/2013,4/19/2013,100.00,N,104(a),47.31(a),http://www.gpo.gov/fdsys/pkg/CFR-2013-title30-...,Dirt Works
8,100.00,000316329,C,8681998,Closed,,100.00,1/28/2013,2/1/2013,4/19/2013,100.00,Y,104(a),56.9300(a),http://www.gpo.gov/fdsys/pkg/CFR-2013-title30-...,Dirt Works
9,100.00,000316329,C,8681996,Closed,,100.00,1/28/2013,2/1/2013,4/19/2013,100.00,N,104(a),56.12028,http://www.gpo.gov/fdsys/pkg/CFR-2013-title30-...,Dirt Works


# Okay, now do it for ALL of the mines

Open up `mines.csv` using pandas and do the same thing, it will just be for more mines this time.

In [95]:
another_new_df = pd.read_csv('mines.csv',dtype ='str')
another_new_df

,id
0,3503598
1,4801789
2,5001797
3,4608254
4,2103723
5,4104757
6,0801306
7,3901432
8,3609624
9,3609931


In [96]:
data_list =[]

def get_violation_info(result):
    driver = webdriver.Chrome()
    driver.get('https://arlweb.msha.gov/drs/drshome.htm')
    # to scroll down
    text_input = driver.find_elements_by_id('inputdrs')[1]
    driver.execute_script("arguments[0].scrollIntoView(true)", text_input)
    text_input.send_keys(result['id'])
    # click search
    driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input').click()

    # get to the search page
    time.sleep(1)
    # enter beginning date
    driver.find_element_by_name('BDate').send_keys('1/1/1995')
    # click "violation"
    driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input').click()
    # get report
    driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input').click()

    # scraping!
    

    rows = driver.find_elements_by_css_selector('tr.drsviols') 

    for row in rows:
        data = {}
        data['violator'] = row.find_elements_by_tag_name('td')[0].text
        data['contract_id'] = row.find_elements_by_tag_name('td')[1].text
        data['citation_no'] = row.find_elements_by_tag_name('td')[2].text
        data['case_no'] = row.find_elements_by_tag_name('td')[3].text
        data['date_issues'] = row.find_elements_by_tag_name('td')[4].text
        data['final_order_date'] = row.find_elements_by_tag_name('td')[5].text
        data['section_of_act'] = row.find_elements_by_tag_name('td')[6].text
        data['date_terminated'] = row.find_elements_by_tag_name('td')[7].text
        data['citation'] = row.find_elements_by_tag_name('td')[8].text
        data['s_and_s'] = row.find_elements_by_tag_name('td')[9].text

        standard = row.find_elements_by_tag_name('td')[10]
        data['standard'] = standard.text
        try:
            data['standard_url'] = standard.find_element_by_tag_name('a').get_attribute('href')
        except:
            data['standard_url'] = 'no data'
        try:
            data['proposed_penalty'] = row.find_elements_by_tag_name('td')[11].text
        except:
            data['proposed_penalty'] ='no data'
        try:
            data['citation_status'] = row.find_elements_by_tag_name('td')[12].text
        except:
            data['citation_status']= 'no data'
        try:
            data['current_penalty'] = row.find_elements_by_tag_name('td')[13].text
        except:
            data['current_penalty'] ='no data'
        try:
            data['amount_paid'] = row.find_elements_by_tag_name('td')[14].text
        except:

            data['amount_paid']='no data'

            
            
        data_list.append(data)
        
    return data_list
    

In [97]:
all_id_df = another_new_df.apply(get_violation_info, axis=1)
pd.DataFrame(all_id_df)

,0
0,"[{'violator': 'Newberg Rock & Dirt', 'contract..."
1,"[{'violator': 'Newberg Rock & Dirt', 'contract..."
2,"[{'violator': 'Newberg Rock & Dirt', 'contract..."
3,"[{'violator': 'Newberg Rock & Dirt', 'contract..."
4,"[{'violator': 'Newberg Rock & Dirt', 'contract..."
5,"[{'violator': 'Newberg Rock & Dirt', 'contract..."
6,"[{'violator': 'Newberg Rock & Dirt', 'contract..."
7,"[{'violator': 'Newberg Rock & Dirt', 'contract..."
8,"[{'violator': 'Newberg Rock & Dirt', 'contract..."
9,"[{'violator': 'Newberg Rock & Dirt', 'contract..."


In [98]:
pd.DataFrame(all_id_df[18])

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,116.00,000440809,C,8992368,Closed,,116.00,5/2/2017,5/2/2017,7/23/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
1,116.00,000435717,C,8992214,Closed,,116.00,2/7/2017,2/8/2017,5/21/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-...,Newberg Rock & Dirt
2,100.00,000398910,C,8872462,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
3,100.00,000398910,C,8872461,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
4,100.00,000383840,C,8790965,Closed,,100.00,4/15/2015,4/15/2015,7/17/2015,100.00,N,104(a),56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
5,100.00,000383840,C,8790964,Closed,,100.00,4/13/2015,4/15/2015,7/17/2015,100.00,N,104(a),56.4101,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
6,100.00,000383840,C,8790962,Closed,,100.00,4/10/2015,4/10/2015,7/17/2015,100.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
7,873.00,000383840,C,8790963,Closed,,873.00,4/10/2015,4/13/2015,7/17/2015,873.00,Y,104(a),56.15020,http://www.gpo.gov/fdsys/pkg/CFR-2015-title30-...,Newberg Rock & Dirt
8,100.00,000345066,C,8694458,Closed,,100.00,1/29/2014,1/29/2014,4/12/2014,100.00,N,104(a),56.12018,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Newberg Rock & Dirt
9,100.00,000345066,C,8694457,Closed,,100.00,1/29/2014,1/29/2014,4/12/2014,100.00,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,Newberg Rock & Dirt
